In [ ]:
# TCC: 
# Code: 
# Author: Anísio Pereira Batista Filho

## Importações

In [25]:
import csv
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import spacy
import string
import time
import tweepy as tw
import unicodedata

from datetime import datetime
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
from itertools import islice
import plotly.express as px
from spellchecker import SpellChecker
from tqdm.auto import tqdm
from unidecode import unidecode
from wordcloud import WordCloud, ImageColorGenerator

## Abertura de arquivo e criação do dataframe

In [26]:
df = pd.read_csv('data/corpus_labeled/iguais/bases_tcc/02_text_preprocessing_base.csv', sep=",", low_memory=False)

In [27]:
df_estados = pd.read_csv("data/utils/abreviações_estados.csv", sep=",", low_memory=True)

In [28]:
dict_estados = df_estados.set_index('sigla')['estado'].to_dict()

In [29]:
df_estados_regioes = pd.read_csv("data/utils/estados_abreviações_regiões.csv", sep=",", low_memory=True)

In [30]:
dict_estados_regioes =  df_estados_regioes.set_index('estado')['região'].to_dict()

## Funções

### Inicializações

In [31]:
tqdm.pandas()

In [32]:
### Twitter Developer Keys:
consumer_key = 'vI9GaSPmhclwCk2e1QzX5noLJ'
consumer_secret = 'Mabmim8tFxVQdgsIPwa6RMXJcvDTq03W2DJLrxH3m1RDwcB2dx'
access_token = '755473300135043072-12XuA8yjOhPJYmIYN82KkZlzC4c8NaJ'
access_token_secret = 'nlpXNfUQFjKoLLJWV0IskOvBjfgr85vwmukxgRK5sdwGH'

### Autgenticação entre Twitter Developer e este script:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)#, wait_on_rate_limit_notify=True)

In [33]:
spell = SpellChecker(language='pt')

In [34]:
geolocator = Nominatim(user_agent="google")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [42]:
nlp = spacy.load('pt_core_news_lg')

### Selecionador de turno

In [87]:
def seleciona_turno(linha):
    linha = pd.to_datetime(linha)
    linha = linha.strftime("%H:%M:%S")
    if linha > '00:00:00' and linha < '11:59:59':
        return 'manhã'
    elif linha > '12:00:00' and linha < '17:59:59':
        return 'tarde'
    else: 
        return 'noite'

### Crawler de usuários

In [12]:
def user_crawler(linha):
    try:
        tweetFetched = api.get_status(linha)
        #Para saber o que poder ser recuperado pelo objeto status ou objeto user, pesquisar.       
        user_screen_name = tweetFetched.user.screen_name
        user_id = tweetFetched.user.id_str
        time.sleep(2)        
    except:
        user_screen_name = 'invalid_user'
        user_id = 'invalid_user'
        
    return user_screen_name, user_id

### Tratamento da coluna de localização

In [13]:
def trata_localizacao(linha):
    ### Convertendo para string
    linha = str(linha)
    ### Removendo os '-'
    linha = re.sub(r'[^\w\s]', ' ', linha)
    ### Removendo a pontuação da string
    linha = linha.translate(str.maketrans('', '', string.punctuation))
    ### Colocando as palavras em caixa baixa
    linha = linha.lower()
    ### Aplicando a correção gramatical do Spellchecker
    linha = spell.correction(linha)
    ### Removendo a acentuação da string
    linha = unidecode(linha)
    ### Removendo espaços múltiplos
    linha = re.sub(r'\s+', ' ', linha)

    for parte in linha.split():
        if parte in dict_estados.keys():
            linha = linha.replace(parte, dict_estados[parte])
    return linha

### Gerador de listas de estados e regiçoes utilizando Geopy

In [14]:
def gera_estado_regiao_geopy(linha):
    try:
        local = geolocator.geocode(linha, addressdetails=True)
        local = local.raw
        local = local['address']

        if local['country_code'] == 'br':
            try:
                if local['state'] in dict_estados_regioes:
                    estado = local['state']
                    regiao = dict_estados_regioes[local['state']]
            except:
                estado = 'statenotdefined'
                regiao = 'regionnotdefined'
        else:
            estado = 'notbrazilstate'
            regiao = 'notbrazilregion'
    except:
        #local = {'state': 'invalid', 'region': 'invalid', 'country': 'invalid'}
        estado = 'invalidstate'
        regiao = 'invalidregion'

    return estado, regiao

### Contador de caracteres

In [88]:
def conta_caracteres(linha):    
    return len(linha)

### Contador de palavras

In [89]:
def conta_palavras(linha):
    return len(linha.split())

### Analisador morfológico

In [90]:
def analisa_morfologia(linha):
    doc = nlp(linha)
    
    substantivos = adjetivos = verbos = adverbios = 0
    for token in doc:
        if token.pos_ == 'NOUN':
            substantivos += 1
        elif token.pos_ == 'ADJ':
            adjetivos += 1
        elif token.pos_ == 'VERB':
            verbos += 1
        elif token.pos_ == 'ADV':
            adverbios += 1
    
    return substantivos, adjetivos, verbos, adverbios

## Main

### Criação de colunas e ajuste de dados

#### Criação da coluna de turnos

In [ ]:
df['time_shift'] = df.created_at.progress_map(seleciona_turno)

#### Criação das colunas de nome de usuário e id de usuário

In [ ]:
df['user_screen_name'], df['user_id'] = zip(*df.tweet_id.progress_apply(lambda x: user_crawler(x)))

#### Preenchimento dos campos em branco da localização dos usuários

In [ ]:
df['user_location'] = df['user_location'].fillna("invalidlocation")

#### Criação da coluna de localização tratada

In [ ]:
df['location_treated'] = df.user_location.progress_map(trata_localizacao)

#### Criação das colunas de Estado e Região com base na localização tratada

In [ ]:
df['state_location'], df['region_location'] = zip(*df.location_treated.progress_apply(lambda x: gera_estado_regiao_geopy(x)))

#### Calculo da quantidade de caracteres do stemming

In [ ]:
df['tweet_text_stemming_char_len'] = df.tweet_text_stemming.progress_map(conta_caracteres)

#### Calculo da quantidade de palavras do stemming

In [ ]:
df['tweet_text_stemming_word_len'] = df.tweet_text_stemming.progress_map(conta_palavras)

#### Calculo da quantidade de caracteres do lemmatization

In [ ]:
df['tweet_text_lemmatization_char_len'] = df.tweet_text_lemmatization.progress_map(conta_caracteres)

#### Calculo da quantidade de palavras do lemmatization

In [ ]:
df['tweet_text_lemmatization_word_len'] = df.tweet_text_lemmatization.progress_map(conta_palavras)

#### Geração de características NAVA para o stemming

In [82]:
df['tweet_text_stemming_noun'], df['tweet_text_stemming_adj'], df['tweet_text_stemming_verb'], df['tweet_text_stemming_adv']  = zip(*df.tweet_text_stemming.progress_apply(lambda x: analisa_morfologia(x)))

  0%|          | 0/3600 [00:00<?, ?it/s]

#### Geração de características NAVA para o lemmatization

In [83]:
df['tweet_text_lemmatization_noun'], df['tweet_text_lemmatization_adj'], df['tweet_text_lemmatization_verb'], df['tweet_text_lemmatization_adv']  = zip(*df.tweet_text_lemmatization.progress_apply(lambda x: analisa_morfologia(x)))

  0%|          | 0/3600 [00:00<?, ?it/s]

## Salvando a base

In [85]:
df.to_csv('data/corpus_labeled/iguais/bases_tcc/03_geracao_carcteristicas_base.csv', sep=',', index=False)

In [86]:
df

,tweet_id,created_at,user_location,tweet_text,label_A,label_B,pct_certainty_A,pct_certainty_B,tweet_text_preproc,tweet_text_stemming,...,tweet_text_lemmatization_char_len,tweet_text_lemmatization_word_len,tweet_text_stemming_noun,tweet_text_stemming_adj,tweet_text_stemming_verb,tweet_text_stemming_adv,tweet_text_lemmatization_noun,tweet_text_lemmatization_adj,tweet_text_lemmatization_verb,tweet_text_lemmatization_adv
0,1346534190335074310,Tue Jan 05 19:09:03 +0000 2021,"São Paulo, Brasil",@HugoMachadoFil3 @quadrelli_marco A parceria c...,-1,-1.0,75,100.0,"['parceria', 'chineses', 'vacina', 'preve', 't...","['parc', 'chines', 'vacin', 'prev', 'troc', 'c...",...,192,18,6,4,3,0,10,2,4,0
1,1349140630753210373,Tue Jan 12 23:46:06 +0000 2021,Brasil,@danielgirardi83 @Fenovais28 @mab_sp125 @samyd...,0,0.0,75,75.0,"['por que', 'deixa', 'minion', 'nervoso', 'jur...","['por qu', 'deix', 'minion', 'nerv', 'jur', 'i...",...,165,16,6,3,1,0,7,3,3,0
2,1348749014964056065,Mon Jan 11 21:49:58 +0000 2021,Barcelona,@edisonrd1977 Es peligrosa la vacuna de Pfizer?,0,0.0,100,100.0,"['espirito santo', 'peligrosa', 'la', 'vacuna'...","['espirito sant', 'peligr', 'la', 'vacun', 'va...",...,57,6,1,1,0,0,2,0,0,0
3,1349129476425314306,Tue Jan 12 23:01:47 +0000 2021,São Paulo,@MBittencourtMD @samydana @GabrielNemer_ @mab_...,-1,-1.0,25,100.0,"['entao', 'perdi', 'aqui', 'grupo', 'placebo',...","['enta', 'perd', 'aqu', 'grup', 'placeb', 'peg...",...,156,17,9,1,2,1,9,0,5,2
4,1348781498602807305,Mon Jan 11 23:59:03 +0000 2021,invalidlocation,"Vidas importam! Sim, vidas importam. Mas tamb...",0,0.0,25,75.0,"['vidas', 'importam', 'sim', 'vidas', 'importa...","['vid', 'import', 'sim', 'vid', 'import', 'rec...",...,158,16,8,2,1,1,9,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,1347323085310078976,Thu Jan 07 23:23:50 +0000 2021,timbaktu,@greenparadyse @gsilvaflavio @CNNBrasil Você s...,0,0.0,25,100.0,"['leu', 'manchete', 'vacina', 'informar', 'sob...","['leu', 'manchet', 'vacin', 'inform', 'sobr', ...",...,73,7,3,1,1,0,3,0,3,0
3596,1347676480717905922,Fri Jan 08 22:48:06 +0000 2021,invalidlocation,"@GugaNoblat Q dó, ver um esquerdinha grudado n...",0,0.0,100,100.0,"['ver', 'esquerdinha', 'grudado', 'bolas', 'do...","['ver', 'esquerd', 'grud', 'bol', 'dor', 'vaci...",...,69,7,4,2,1,0,3,1,3,0
3597,1348598899896946690,Mon Jan 11 11:53:28 +0000 2021,São Paulo,@da_cia @victormasc Não tenho nada a ver com n...,1,1.0,75,100.0,"['nada', 'ver', 'nada', 'disso', 'sequer', 'vo...","['nad', 'ver', 'nad', 'diss', 'sequ', 'vot', '...",...,197,22,7,2,1,0,9,0,7,0
3598,1347313773938434052,Thu Jan 07 22:46:50 +0000 2021,invalidlocation,"@djivanrodrigues @VEJA Sim, mas antes ele sina...",0,0.0,25,100.0,"['sim', 'antes', 'sinalizou', 'apenas', 'milho...","['sim', 'ant', 'sinaliz', 'apen', 'milho', 'va...",...,180,18,14,2,0,1,7,1,5,3
